In [1]:
import argparse
from argparse import Namespace

import os.path

import numpy as np
import argparse
import nibabel as nib

from scipy import signal
from torchmetrics import MeanSquaredError
from viu.io import volume
from viu.torch.deformation.fields import DVF
from viu.torch.math import torch_affine_to_vol_mat
from viu.util.body_mask import seg_body

from pamomo.registration.deformable import reg
import torch
import torch.nn.functional as tt

from exampleUtils import *

In [2]:
args = Namespace(\
    src=f'/home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/Patient0119826_movingScan_14.nii.gz',\
    dst=f'/home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/Patient0119826_fixedScan_22.nii.gz',\
    body_seg=True,\
    air_threshold=-300,\
    alpha=50,\
    verboseMode=False,\
    maxNumberOfIterations=100,\
    numLevels=3,\
    saveMovedAsNIIFlag=True)
print(args)

Namespace(src='/home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/Patient0119826_movingScan_14.nii.gz', dst='/home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/Patient0119826_fixedScan_22.nii.gz', body_seg=True, air_threshold=-300, alpha=50, verboseMode=False, maxNumberOfIterations=100, numLevels=3, saveMovedAsNIIFlag=True)


In [3]:
doHUScaling=True #Select this based on whether using 5DCT data or not.
makeUINT16ForDefReg=True
hu_offset=-1000 
hu_max=400
def huScaleVol(vol,hu_offset, hu_max, makeUINT16ForDefReg):
    scanVol = vol.astype('float32')
    scanVol_min, scanVol_max = np.min(scanVol), np.max(scanVol)
    #First scale volume to 0-1 (if not already)
    vol_data_ZYX, volMin, volMax = scanVol, scanVol_min, scanVol_max
    volScaled_data_ZYX = (vol_data_ZYX - volMin) / (volMax - volMin)
    #Then scale to hu_min to hu_max
    hu_min=hu_offset
    vol_ZYX_HU = hu_min + volScaled_data_ZYX * (hu_max -hu_min)
    scanVol, scanVol_min, scanVol_max = vol_ZYX_HU, np.min(vol_ZYX_HU), np.max(vol_ZYX_HU)

    if True==makeUINT16ForDefReg:
        if scanVol_min < 0:
            scanVol = (vol_ZYX_HU-scanVol_min).astype('uint16') 
            scanVol_min, scanVol_max = np.min(scanVol), np.max(scanVol)

    return scanVol, scanVol_min, scanVol_max

In [4]:
#registrationCommand(args)
dst_path = os.path.dirname(args.dst)

src_vol: np.ndarray
if args.src.endswith('.nii.gz') and args.dst.endswith('.nii.gz'):
    src_vol, src_res, src_pos = ImportFromNII(args.src)
    dst_vol, dst_res, dst_pos = ImportFromNII(args.dst)
else:
    src_vol, src_res, src_pos = volume.read_volume(args.src)
    dst_vol, dst_res, dst_pos = volume.read_volume(args.dst)

print(f'src_vol dtype {src_vol.dtype}, min Val {np.min(src_vol)} max Val {np.max(src_vol)} shape ZYX {src_vol.shape}, res XYZ {src_res}, pos XYZ {src_pos}')
# src_vol dtype float64,  shape ZYX (590, 512, 512), res XYZ [0.9765625 0.9765625 1.       ], pos XYZ [-249.50999451 -469.51000977 -352.3999939 ]
print(f'dst_vol dtype {dst_vol.dtype}, min Val {np.min(src_vol)} max Val {np.max(src_vol)} shape ZYX {dst_vol.shape}, res XYZ {dst_res}, pos XYZ {dst_pos}')
# dst_vol dtype float64,  shape ZYX (590, 512, 512), res XYZ [0.9765625 0.9765625 1.       ], pos XYZ [-249.50999451 -469.51000977 -352.3999939 ]

src_vol dtype float64, min Val 0.0 max Val 1.0 shape ZYX (256, 256, 256), res XYZ [2. 2. 2.], pos XYZ [-249.5 -544.5 -971. ]
dst_vol dtype float64, min Val 0.0 max Val 1.0 shape ZYX (256, 256, 256), res XYZ [2. 2. 2.], pos XYZ [-249.5 -544.5 -971. ]


In [5]:
if True==doHUScaling:
    src_vol, src_vol_min, src_vol_max =huScaleVol(src_vol, hu_offset, hu_max, makeUINT16ForDefReg)
    print(f'src_vol dtype {src_vol.dtype}, min Val {np.min(src_vol)} = {src_vol_min}  max Val {np.max(src_vol)} = {src_vol_max}')
    dst_vol, dst_vol_min, dst_vol_max =huScaleVol(dst_vol, hu_offset, hu_max, makeUINT16ForDefReg)
    print(f'dst_vol dtype {dst_vol.dtype}, min Val {np.min(dst_vol)} = {dst_vol_min}  max Val {np.max(dst_vol)} = {dst_vol_max}')

src_vol dtype uint16, min Val 0 = 0  max Val 1400 = 1400
dst_vol dtype uint16, min Val 0 = 0  max Val 1400 = 1400


In [6]:
runFreshRegistration=False
cachedPath = os.path.join(dst_path, f'Patient0119826_cachedDVF.npz')
if True==runFreshRegistration:

    additional_args = {}
    if args.body_seg:
        print('Find connected components...')
        #Find connected components...
        msk = seg_body(dst_vol, air_threshold=args.air_threshold)
        additional_args.update({'dst_seg': {'SM_bodymask': msk}, 'similarityMaskMultilevelStrategy': 'STRICTINTERIOR'})

    print('Start registration...')
    vol_min = -1200
    dvf, dvf_res, dvf_pos = reg(src_vol.clip(min=vol_min), src_res,
                                dst_vol.clip(min=vol_min), dst_res,
                                alpha=args.alpha,#alpha=20,
                                numLevels=args.numLevels,
                                finestLevelReference=0,
                                finestLevelTemplate=0,
                                maxNumberOfIterations=args.maxNumberOfIterations,
                                **additional_args) #verboseMode=args.verboseMode, **additional_args #verboseMode='true', **additional_args
    np.savez_compressed(cachedPath, dvf, dvf_res, dvf_pos)
    # Start registration...

    # This is the Fraunhofer MEVIS cuda registration library. Version: VarianDeformableRegistrationDLLCUDA -- v1.6.2, built Dec 11 2023, 20:49:53
    # numSourceSegments <= 0. Running without mask alignment. Continuing.
    # Started registration...
    # Optimization on level 1 / 3 needs: 3.39014 s
    # Optimization on level 2 / 3 needs: 8.03293 s
    # Optimization on level 3 / 3 needs: 84.1789 s
    # Finished multilevel registration.
    # Total runtime: 103.207s.
    # Done.
else:
    cachedDVF = np.load(cachedPath)
    dvf, dvf_res, dvf_pos = cachedDVF['arr_0'], cachedDVF['arr_1'], cachedDVF['arr_2']

print(f'B4 DVFify dvf type {type(dvf)}  dtype {dvf.dtype},  shape ZYX {dvf.shape}')
# B4 DVFify dvf type <class 'numpy.ndarray'>  dtype float32,  shape ZYX (295, 256, 256, 3)
print(f'B4 tensorify dvf_res dtype {dvf_res.dtype},  shape {dvf_res.shape}, value XYZ  {dvf_res}')
# B4 tensorify dvf_res dtype float64,  shape (3,), value XYZ  [1.95695466 1.95695466 2.00340136]
print(f'B4 tensorify dvf_pos dtype {dvf_pos.dtype},  shape {dvf_pos.shape}, value XYZ  {dvf_pos}')
# B4 tensorify dvf_pos dtype float64,  shape (3,), value XYZ  [0. 0. 0.]

B4 DVFify dvf type <class 'numpy.ndarray'>  dtype float32,  shape ZYX (256, 256, 256, 3)
B4 tensorify dvf_res dtype float64,  shape (3,), value XYZ  [2. 2. 2.]
B4 tensorify dvf_pos dtype float64,  shape (3,), value XYZ  [0. 0. 0.]


In [7]:
dvf_res = torch.tensor(dvf_res, dtype=torch.float64)
print(f'After tensorify dvf_res dtype {dvf_res.dtype},  shape {dvf_res.shape}, value XYZ  {dvf_res}')
dvf_pos = torch.tensor(dvf_pos, dtype=torch.float64)
print(f'After tensorify dvf_pos dtype {dvf_pos.dtype},  shape {dvf_pos.shape}, value XYZ  {dvf_pos}')

dst_dim = torch.tensor(dst_vol.shape[::-1]) #From ZYX to XYZ
print(f'dst_dim in XYZ dtype {dst_dim.dtype},  shape {dst_dim.shape}, value XYZ  {dst_dim}')
dst_res = torch.tensor(dst_res, dtype=torch.float64)
print(f'dst_res in XYZ dtype {dst_res.dtype},  shape {dst_res.shape}, value XYZ  {dst_res}')

After tensorify dvf_res dtype torch.float64,  shape torch.Size([3]), value XYZ  tensor([2., 2., 2.], dtype=torch.float64)
After tensorify dvf_pos dtype torch.float64,  shape torch.Size([3]), value XYZ  tensor([0., 0., 0.], dtype=torch.float64)
dst_dim in XYZ dtype torch.int64,  shape torch.Size([3]), value XYZ  tensor([256, 256, 256])
dst_res in XYZ dtype torch.float64,  shape torch.Size([3]), value XYZ  tensor([2., 2., 2.], dtype=torch.float64)


In [8]:
#volume tensor with batch and channel
src_vol_tensor = torch.from_numpy(src_vol.astype('float32')).unsqueeze(0).unsqueeze(0)
printTensor("src_vol_tensor", src_vol_tensor)

src_vol_tensor shape: torch.Size([1, 1, 256, 256, 256]) device: cpu dtype: torch.float32


In [9]:
#Create batched DVF object
# dvf = DVF(dvf).from_millimeter(dvf_res).to(torch.float32)
# Above line gives error: DVFs can be formed only of tensors of shape (B,H,W,2) or (B,D,H,W,3). Provided shape: torch.Size([295, 256, 256, 3]).
batched_dvf = DVF(dvf[None,...]).from_millimeter(dvf_res).to(torch.float32)
print(f'batched_dvf DVF object   dtype {batched_dvf.dtype},  shape ZYX3 {batched_dvf.shape}')

batched_dvf DVF object   dtype torch.float32,  shape ZYX3 torch.Size([1, 256, 256, 256, 3])


In [10]:
#_call_ method => pull warping, no prefilter
# warped_vol_F_noPreFilter = batched_dvf1(vol_tensor).squeeze(0).squeeze(0).numpy()
#Instead of call method use explicit sample method
# warped_vol_F_noPreFilter = batched_dvf1.sample(vol_tensor, mode="bilinear", padding_mode="zeros", warpingModeString="pull",prefilter=False).squeeze(0).squeeze(0).numpy()
warped_vol_F_noPreFilter = batched_dvf.sample(src_vol_tensor, mode="bilinear", padding_mode="zeros", warpingModeString="pull",prefilter=False).squeeze(0).squeeze(0).numpy()
print(f'warped_vol_F_noPreFilter  object   dtype {warped_vol_F_noPreFilter.dtype},  shape ZYX3 {warped_vol_F_noPreFilter.shape}')

# #Display
v1_volumeComparisonViewer3D(
    listVolumes=[src_vol, dst_vol, warped_vol_F_noPreFilter],listLabels=['src', 'dst', 'warped_vol_F_noPreFilter'],
    maxZ0=src_vol.shape[0], maxZ1=src_vol.shape[1], maxZ2=src_vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=0, wMax=1400)

v1_volumeComparisonViewer3D(
    listVolumes=[src_vol-dst_vol, warped_vol_F_noPreFilter-dst_vol],listLabels=['F-M', 'M*_NoPreFilter-M'],
    maxZ0=src_vol.shape[0], maxZ1=src_vol.shape[1], maxZ2=src_vol.shape[2],
    figsize=(12,8), cmap='coolwarm',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-100, wMax=100)

warped_vol_F_noPreFilter  object   dtype float32,  shape ZYX3 (256, 256, 256)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [11]:
#_call_ method => pull warping, with  prefilter <- interpol behaviour
# Instead of call method use explicit sample method
warped_vol_interpol_preFilter = batched_dvf.sample(src_vol_tensor, mode="cubic", padding_mode="zeros", warpingModeString="pull",prefilter=True).squeeze(0).squeeze(0).numpy()
print(f'warped_vol_interpol_preFilter  object   dtype {warped_vol_interpol_preFilter.dtype},  shape ZYX3 {warped_vol_interpol_preFilter.shape}')

#Display
v1_volumeComparisonViewer3D(
    listVolumes=[src_vol, dst_vol, warped_vol_interpol_preFilter],listLabels=['src', 'dst', 'warped_vol_interpol_preFilter'],
    maxZ0=src_vol.shape[0], maxZ1=src_vol.shape[1], maxZ2=src_vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=0, wMax=1400)

v1_volumeComparisonViewer3D(
    listVolumes=[src_vol-dst_vol, warped_vol_interpol_preFilter-dst_vol],listLabels=['F-M', 'M*_PreFilter-M'],
    maxZ0=src_vol.shape[0], maxZ1=src_vol.shape[1], maxZ2=src_vol.shape[2],
    figsize=(12,8), cmap='coolwarm',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-100, wMax=100)

/opt/conda/envs/clone2MMIPython/lib/python3.9/site-packages/interpol/autograd.py:287: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/third_party/nvfuser/csrc/parser.cpp:3820.)
  output = spline_coeff_nd(input, *opt)


warped_vol_interpol_preFilter  object   dtype float32,  shape ZYX3 (256, 256, 256)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [12]:
#Display the difference between torch-functional output and torch-interpol output
v1_volumeComparisonViewer3D(
    listVolumes=[warped_vol_interpol_preFilter - warped_vol_F_noPreFilter],listLabels=['M*_PreFilter - M*_NoPreFilter'],
    maxZ0=src_vol.shape[0], maxZ1=src_vol.shape[1], maxZ2=src_vol.shape[2],
    figsize=(12,8), cmap='coolwarm',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-50, wMax=50)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

##### Reproduce prevention of smoothing during interpol based warping

In [13]:
# experimentFolder = pathlib.Path('/mnt/data/supratik/diaphragm_detection/data/')
experimentFolder = pathlib.Path('/home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/')
patientMRN='Patient04PB_bin_01'
dcmFolder = experimentFolder / patientMRN
vol, res, pos = read_volume(str(dcmFolder))
print(f'dcmFolder {dcmFolder}')
print(f'vol type {(type(vol))} shape {vol.shape} min {np.min(vol)} max {np.max(vol)} ')
print(f'res type {type(res)} value {res} ')
print(f'pos type {type(pos)} value {pos}')
toy_depth, toy_height, toy_width = vol.shape[0], vol.shape[1], vol.shape[2]
#Make the dimensions unequal
org_width = toy_width
# toy_width = 8*toy_width//10
# print(f'toy_depth {toy_depth}, toy_height {toy_height}, toy_width {toy_width}, org_width {org_width}')
vol = vol[:,:,:toy_width]
print(f'vol type {(type(vol))} shape {vol.shape}')
#With truncation in X direction, the pos matching the center of the volume in patient co-ordinate also shifts
old_pos_X = pos[0]
res_X = res[0]
new_pos_X = old_pos_X - 0.5*(org_width - toy_width)*res_X
pos[0] = new_pos_X
print(f'pos type {type(pos)} value {pos}')


dcmFolder /home/wd974888/Downloads/workingFolder/DeformationExperiment/readFormat/Patient04PB_bin_01
vol type <class 'numpy.ndarray'> shape (210, 512, 512) min -1364.0 max 1915.0 
res type <class 'numpy.ndarray'> value [0.9765625 0.9765625 1.       ] 
pos type <class 'numpy.ndarray'> value [ 1.71875000e-03 -2.18498281e+02 -1.96400000e+02]
vol type <class 'numpy.ndarray'> shape (210, 512, 512)
pos type <class 'numpy.ndarray'> value [ 1.71875000e-03 -2.18498281e+02 -1.96400000e+02]


In [14]:
v1_volumeComparisonViewer3D(
    listVolumes=[vol],listLabels=['original'],
    maxZ0=vol.shape[0], maxZ1=vol.shape[1], maxZ2=vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [15]:
device=torch.device('cuda:0')
batchSize=1

In [16]:
#volume tensor with batch and channel
vol_tensor = torch.from_numpy(vol).to(device).unsqueeze(0).unsqueeze(0)
printTensor("vol_tensor", vol_tensor)

vol_tensor shape: torch.Size([1, 1, 210, 512, 512]) device: cuda:0 dtype: torch.float32


In [17]:
# Rotation around Dicom Z axis : As viewed into Axial plane    from origin toward +Z axis (towards H(Head)), with A (Anterior) on top of the view, pass clockwise rotation is positive
# Rotation around Dicom Y axis : As viewed into Coronal plane  from origin toward +Y axis (towards P(Posterior)), with H (Head) on top of the view, pass clockwise rotation is positive
# Rotation around Dicom X axis : As viewed  (lying down horizontally) into Sagittal plane from +X toward origin (towards R(Right)) with H (Head) on top of the view, pass clockwise rotation is positive
theta_deg = 30.00 #Rotate image  counter clockwise by 10 degree
viewString='axial'
pushAffine_np = getPushRotationMatrix(
    theta_deg=theta_deg,
    viewString=viewString,
    center_slice_z=toy_depth/2.0,
    center_row_y=toy_height/2.0,
    center_col_x=toy_width/2.0)
print(f'pushAffine_np {pushAffine_np}')
pullAffine_np = np.linalg.inv(pushAffine_np)
print(f'pullAffine_np {pullAffine_np}')

pushAffine_np [[  1.          0.          0.          0.       ]
 [  0.          0.8660254   0.5       -93.702515 ]
 [  0.         -0.5         0.8660254 162.2975   ]
 [  0.          0.          0.          1.       ]]
pullAffine_np [[  1.          0.          0.          0.       ]
 [  0.          0.8660254  -0.5       162.29752  ]
 [  0.          0.5         0.8660254 -93.7025   ]
 [  0.          0.          0.          1.       ]]


In [18]:
pyTorchPullAffineMat3x4 = getPyTorchAffineMatTensor(pullAffine_np,  toy_depth, toy_height, toy_width, toy_depth, toy_height, toy_width, device)
printTensor('pyTorchPullAffineMat3x4', pyTorchPullAffineMat3x4)
print(f'pyTorchPullAffineMat3x4 {pyTorchPullAffineMat3x4}')

pyTorchPullAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
pyTorchPullAffineMat3x4 tensor([[ 8.6603e-01,  5.0000e-01,  0.0000e+00, -5.9605e-08],
        [-5.0000e-01,  8.6603e-01,  0.0000e+00,  5.9605e-08],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00]], device='cuda:0')


In [19]:
# from viu.torch.deformation.fields import DVF
printTensor('pyTorchPullAffineMat3x4', pyTorchPullAffineMat3x4)
nb_dim = pyTorchPullAffineMat3x4.shape[-1]-1
viuDVFGridSizeParam=(batchSize, 1, toy_depth, toy_height, toy_width, nb_dim)
print(f'viuDVFGridSizeParam shape: (batch, channel, D, H, W, nbDim=3): {viuDVFGridSizeParam}')

#While calling DVF.affine, in the affine matrix, the last row (0,0,1) for 2D or (0,0,0,1) for is removed. However no batch is added.
#Also  while passing the size, the  DVF  dimension  (2 or 3) is being passed  as the last elemet of the size  tuple.
#Further  line #393 of field.py  is to be modified.
#Another important difference  is that  the DVF returned by DVF.affine  includes a subtraction of the identity map
viu_grid_pull = DVF.affine(pyTorchPullAffineMat3x4, size=viuDVFGridSizeParam)
printTensor('viu_grid_pull', viu_grid_pull)

pyTorchPullAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
viuDVFGridSizeParam shape: (batch, channel, D, H, W, nbDim=3): (1, 1, 210, 512, 512, 3)


viu_grid_pull shape: torch.Size([1, 210, 512, 512, 3]) device: cuda:0 dtype: torch.float32


In [20]:
printTensor('viu_grid_pull', viu_grid_pull)
printTensor('vol_tensor', vol_tensor)

viu_grid_pull shape: torch.Size([1, 210, 512, 512, 3]) device: cuda:0 dtype: torch.float32
vol_tensor shape: torch.Size([1, 1, 210, 512, 512]) device: cuda:0 dtype: torch.float32


In [21]:
#Test rotation
aVol_viu_grid_pull_warped = viu_grid_pull.sample(
    input=vol_tensor,
    mode='bilinear',
    padding_mode='zeros',
    warpingModeString="pull",
    prefilter=False)
printTensor('aVol_viu_grid_pull_warped', aVol_viu_grid_pull_warped)
aVol_viu_grid_pull_warped_np = aVol_viu_grid_pull_warped.squeeze().squeeze().clone().cpu().numpy()
v1_volumeComparisonViewer3D(
    listVolumes=[aVol_viu_grid_pull_warped_np],listLabels=[f'VIUDVF_PullWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
    maxZ0=aVol_viu_grid_pull_warped_np.shape[0], maxZ1=aVol_viu_grid_pull_warped_np.shape[1], maxZ2=aVol_viu_grid_pull_warped_np.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)

aVol_viu_grid_pull_warped shape: torch.Size([1, 1, 210, 512, 512]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [22]:
numRotations = int(360 // theta_deg)
print(f'numRotations {numRotations}')

numRotations 12


In [23]:
#To demonstrate blurring and its avoidance, attempt multiple  with and without prefilter
test_vol_tensor_noPreF = vol_tensor.clone()
test_vol_tensor_withPreF = vol_tensor.clone()

In [24]:
for count in range(numRotations):
    test_vol_tensor_noPreF = viu_grid_pull.sample(
        input=test_vol_tensor_noPreF,
        mode='bilinear',
        padding_mode='zeros',
        warpingModeString="pull",
        prefilter=False)
printTensor('test_vol_tensor_noPreF', test_vol_tensor_noPreF)
test_vol_tensor_noPreF_np = test_vol_tensor_noPreF.squeeze().squeeze().clone().cpu().numpy()
v1_volumeComparisonViewer3D(
    listVolumes=[test_vol_tensor_noPreF_np],listLabels=[f'VIUDVF_F+NoPreF_Warp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
    maxZ0=test_vol_tensor_noPreF_np.shape[0], maxZ1=test_vol_tensor_noPreF_np.shape[1], maxZ2=test_vol_tensor_noPreF_np.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)

test_vol_tensor_noPreF shape: torch.Size([1, 1, 210, 512, 512]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [25]:
for count in range(numRotations):
    test_vol_tensor_withPreF = viu_grid_pull.sample(
        input=test_vol_tensor_withPreF,
        mode='cubic',
        padding_mode='zeros',
        warpingModeString="pull",
        prefilter=True)
printTensor('test_vol_tensor_withPreF', test_vol_tensor_withPreF)
test_vol_tensor_withPreF_np = test_vol_tensor_withPreF.squeeze().squeeze().clone().cpu().numpy()
v1_volumeComparisonViewer3D(
    listVolumes=[test_vol_tensor_withPreF_np],listLabels=[f'VIUDVF_Interpol+SplinePreF_Warp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
    maxZ0=test_vol_tensor_withPreF_np.shape[0], maxZ1=test_vol_tensor_withPreF_np.shape[1], maxZ2=test_vol_tensor_withPreF_np.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)


/opt/conda/envs/clone2MMIPython/lib/python3.9/site-packages/interpol/autograd.py:287: UserWarning: operator() profile_node %258 : int[] = prim::profile_ivalue(%permutation.21)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  output = spline_coeff_nd(input, *opt)


test_vol_tensor_withPreF shape: torch.Size([1, 1, 210, 512, 512]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [26]:
v1_volumeComparisonViewer3D(
    listVolumes=[test_vol_tensor_noPreF_np-vol, test_vol_tensor_withPreF_np-vol],listLabels=[f'M*-M without preF', f'M*-M with preF'],
    maxZ0=vol.shape[0], maxZ1=vol.shape[1], maxZ2=vol.shape[2],
    figsize=(12,8), cmap='coolwarm',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-50, wMax=50, useAspectCol=False)

interactive(children=(Output(),), _dom_classes=('widget-interact',))